In [ ]:
#C2a
def sendPartition(iter):
    client = MongoClient()
    db = client.TaskC_db
    climate = db.ClimateData
    fire = db.FireData
        
    for line in iter:
        datatype = line[0]
        if datatype == "cdata":
            Climate={}
            Climate["Station"] = int(line[1])
            Climate["Date"] = line[9]
            Climate["Air Temperature(Celcius)"] = float(line[2])
            Climate["Relative Humidity"] = float(line[3])
            Climate["WindSpeed  (knots)"] = float(line[4])
            Climate["Max Wind Speed"] = float(line[5])
            Climate["MAX"] = line[6]
            Climate["MIN"] = line[7]
            Climate["Precipitation"] = line[8]
            Climate["fire"] = []
            climate.insert_one(Climate)
                
        if datatype == "fdata":
            Fire={}
            Fire["Latitude"] = float(line[1])
            Fire["Longitude"] = float(line[2])
            Fire["Surface Temperature (kelvin)"] = float(line[3])
            Fire["Datetime"] = line[8]
            Fire["Power"] = float(line[4])
            Fire["Confidence"] = float(line[5])
            Fire["Date"] = line[7]
            Fire["Surface Temperature (Celcius)"] = float(line[6])
            fire.insert_one(Fire)
            
            climate.update_one({"Date":Fire["Date"]},
                               {"$push":{ "fire": {"_id": Fire["_id"]}}})           

In [1]:
import pymongo
from pymongo import MongoClient
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

    
sc = SparkContext.getOrCreate()

if (sc is None):
    sc = SparkContext("local[2]","FireClimate")
ssc = StreamingContext(sc, 5)

host = "localhost"
port = 9999

lines = ssc.socketTextStream(host, int(port))
Line = lines.map(lambda line: line.split(","))                          
Line.pprint()
Line.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))

ssc.start()
try:
    ssc.awaitTermination(timeout=800)
except KeyboardInterrupt:
    ssc.stop()
    sc.stop()

ssc.stop()
sc.stop()

-------------------------------------------
Time: 2018-06-06 21:39:40
-------------------------------------------
['cdata', '948700', '19', '56.8', '7.9', '11.1', '72.0*', '61.9*', '0.00I', '2018-06-06 21:39:39']
['fdata', '-37.966', '145.051', '341.8', '26.7', '78', '68', '2018-06-06 21:39:39', '2018-06-06 21:39:39.758955']
['fdata', '-35.541', '143.311', '336.3', '62', '82', '63', '2018-06-06 21:39:39', '2018-06-06 21:39:39.759152']
['fdata', '-35.554', '143.307', '326.8', '23.8', '67', '53', '2018-06-06 21:39:39', '2018-06-06 21:39:39.759452']
['fdata', '-35.543', '143.316', '340.4', '84.2', '86', '67', '2018-06-06 21:39:39', '2018-06-06 21:39:39.759572']
['fdata', '-37.708', '145.1', '327.8', '16.2', '80', '54', '2018-06-06 21:39:39', '2018-06-06 21:39:39.760113']



Py4JJavaError: An error occurred while calling o21.awaitTerminationOrTimeout.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/Users/zhangbaofang/anaconda3/lib/python3.6/site-packages/pyspark/streaming/util.py", line 65, in call
    r = self.func(t, *rdds)
  File "/Users/zhangbaofang/anaconda3/lib/python3.6/site-packages/pyspark/streaming/dstream.py", line 159, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "<ipython-input-1-755b66818153>", line 19, in <lambda>
    Line.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))
NameError: name 'sendPartition' is not defined

	at org.apache.spark.streaming.api.python.TransformFunction.callPythonTransformFunction(PythonDStream.scala:95)
	at org.apache.spark.streaming.api.python.TransformFunction.apply(PythonDStream.scala:78)
	at org.apache.spark.streaming.api.python.PythonDStream$$anonfun$callForeachRDD$1.apply(PythonDStream.scala:179)
	at org.apache.spark.stre

-------------------------------------------
Time: 2018-06-06 21:39:45
-------------------------------------------
['cdata', '948700', '15', '50.7', '9.2', '13', '', '58.3', '0.02G', '2018-06-06 21:39:40']
['fdata', '-35.646', '142.282', '305.6', '11.8', '65', '32', '2018-06-06 21:39:40', '2018-06-06 21:39:40.761284']
['fdata', '-38.075', '143.76', '320.1', '17.6', '72', '46', '2018-06-06 21:39:40', '2018-06-06 21:39:40.761384']
['fdata', '-37.636', '149.33', '316.7', '37.1', '94', '43', '2018-06-06 21:39:40', '2018-06-06 21:39:40.761469']
['fdata', '-37.624', '149.332', '306.8', '20.8', '69', '33', '2018-06-06 21:39:40', '2018-06-06 21:39:40.761552']
['fdata', '-37.82', '142.323', '327.6', '18.9', '62', '54', '2018-06-06 21:39:40', '2018-06-06 21:39:40.761632']
['cdata', '948700', '16', '53.6', '8.1', '15', '68.4*', '54.3*', '0.00G', '2018-06-06 21:39:41']
['fdata', '-37.875', '142.51', '346.2', '46', '93', '73', '2018-06-06 21:39:41', '2018-06-06 21:39:41.765834']
['fdata', '-37.608',

-------------------------------------------
Time: 2018-06-06 21:40:20
-------------------------------------------
['cdata', '948700', '21', '64.5', '10.6', '15.9', '86.4*', '59.4*', '1.26G', '2018-06-06 21:40:14']
['fdata', '-37.376', '148.042', '321.2', '17.3', '81', '48', '2018-06-06 21:40:14', '2018-06-06 21:40:14.963048']
['fdata', '-37.375', '148.054', '322.6', '18.2', '82', '49', '2018-06-06 21:40:14', '2018-06-06 21:40:14.963790']
['fdata', '-37.386', '148.043', '328.4', '25.9', '84', '55', '2018-06-06 21:40:14', '2018-06-06 21:40:14.964172']
['fdata', '-37.384', '148.056', '336.5', '38.3', '90', '63', '2018-06-06 21:40:14', '2018-06-06 21:40:14.964405']
['fdata', '-37.367', '148.04', '328.5', '24.3', '71', '55', '2018-06-06 21:40:14', '2018-06-06 21:40:14.964712']
['cdata', '948700', '16', '48.1', '9.3', '12', '68.2*', '57.9*', '0.00G', '2018-06-06 21:40:15']
['fdata', '-37.471', '148.091', '323.4', '21.3', '71', '50', '2018-06-06 21:40:15', '2018-06-06 21:40:15.970607']
['fdat

-------------------------------------------
Time: 2018-06-06 21:40:55
-------------------------------------------
['cdata', '948710', '21', '58.1', '5.3', '11.1', '78.6*', '60.3*', '0.00G', '2018-06-06 21:40:50']
['fdata', '-36.3096', '142.7722', '333.1', '42.8', '85', '59', '2018-06-06 21:40:50', '2018-06-06 21:40:50.151441']
['fdata', '-36.2229', '143.4245', '314.3', '18.7', '61', '41', '2018-06-06 21:40:50', '2018-06-06 21:40:50.152210']
['fdata', '-36.0279', '146.0244', '311.9', '16.7', '52', '38', '2018-06-06 21:40:50', '2018-06-06 21:40:50.152651']
['fdata', '-37.1405', '144.1375', '327.8', '37.5', '81', '54', '2018-06-06 21:40:50', '2018-06-06 21:40:50.153021']
['fdata', '-37.7052', '144.6926', '319.7', '27.3', '72', '46', '2018-06-06 21:40:50', '2018-06-06 21:40:50.153449']
['cdata', '948710', '20', '62.2', '7.1', '15', '75.9*', '59.2', '0.00I', '2018-06-06 21:40:51']
['fdata', '-36.2127', '141.4938', '324.6', '31.2', '78', '51', '2018-06-06 21:40:51', '2018-06-06 21:40:51.1547

...

-------------------------------------------
Time: 2018-06-06 21:42:40
-------------------------------------------
['cdata', '948720', '11', '44.2', '8.7', '15.9', '55.8*', '47.5', '0.04G', '2018-06-06 21:42:35']
['fdata', '-36.4057', '140.9806', '392.2', '214.4', '100', '119', '2018-06-06 21:42:35', '2018-06-06 21:42:35.679653']
['fdata', '-36.3815', '143.8529', '318.6', '18.8', '58', '45', '2018-06-06 21:42:35', '2018-06-06 21:42:35.679916']
['fdata', '-36.9455', '141.849', '324.2', '27.1', '77', '51', '2018-06-06 21:42:35', '2018-06-06 21:42:35.680499']
['fdata', '-36.4384', '141.7444', '328.4', '32.9', '81', '55', '2018-06-06 21:42:35', '2018-06-06 21:42:35.680722']
['fdata', '-36.3711', '145.8311', '313.6', '14.4', '59', '40', '2018-06-06 21:42:35', '2018-06-06 21:42:35.681079']
['cdata', '948720', '10', '45.7', '3.6', '7', '54.0*', '45.5*', '0.01G', '2018-06-06 21:42:36']
['fdata', '-36.3268', '142.3643', '312.1', '12.9', '53', '38', '2018-06-06 21:42:36', '2018-06-06 21:42:36.685

-------------------------------------------
Time: 2018-06-06 21:43:15
-------------------------------------------
['cdata', '948720', '15', '41.5', '17', '28.9', '64.2*', '47.3', '0.00I', '2018-06-06 21:43:09']
['fdata', '-36.9318', '143.0907', '396', '236', '100', '122', '2018-06-06 21:43:09', '2018-06-06 21:43:09.855140']
['fdata', '-36.3195', '146.0601', '350.3', '72.5', '83', '77', '2018-06-06 21:43:09', '2018-06-06 21:43:09.855395']
['fdata', '-36.5839', '143.0588', '372.9', '132.6', '100', '99', '2018-06-06 21:43:09', '2018-06-06 21:43:09.855658']
['fdata', '-36.5854', '143.0462', '367.4', '114.5', '100', '94', '2018-06-06 21:43:09', '2018-06-06 21:43:09.856184']
['fdata', '-36.3863', '143.339', '335.5', '42.3', '82', '62', '2018-06-06 21:43:09', '2018-06-06 21:43:09.856324']
['cdata', '948720', '9', '42', '4.6', '12', '60.3*', '41.4*', '0.00I', '2018-06-06 21:43:10']
['fdata', '-36.3014', '143.203', '350.1', '69.9', '95', '76', '2018-06-06 21:43:10', '2018-06-06 21:43:10.862111'

-------------------------------------------
Time: 2018-06-06 21:43:50
-------------------------------------------
['cdata', '948720', '17', '38.7', '16.8', '22.9', '65.1*', '48.9', '0.00I', '2018-06-06 21:43:45']
['fdata', '-36.1581', '142.7434', '316.2', '10.6', '65', '43', '2018-06-06 21:43:45', '2018-06-06 21:43:45.050292']
['fdata', '-36.9045', '141.9823', '386.7', '181.2', '100', '113', '2018-06-06 21:43:45', '2018-06-06 21:43:45.050374']
['fdata', '-37.6593', '142.6655', '330', '30.3', '83', '56', '2018-06-06 21:43:45', '2018-06-06 21:43:45.050443']
['fdata', '-36.2851', '145.8', '337.5', '37.7', '88', '64', '2018-06-06 21:43:45', '2018-06-06 21:43:45.050511']
['fdata', '-36.5577', '145.3929', '337.5', '40.6', '88', '64', '2018-06-06 21:43:45', '2018-06-06 21:43:45.050552']
['cdata', '948720', '12', '41', '12.9', '16.9', '59.4*', '48.6*', '0.16G', '2018-06-06 21:43:46']
['fdata', '-37.7751', '143.0494', '334.6', '37.1', '86', '61', '2018-06-06 21:43:46', '2018-06-06 21:43:46.0553

-------------------------------------------
Time: 2018-06-06 21:44:25
-------------------------------------------
['cdata', '948720', '17', '46.4', '9.5', '20', '71.8*', '42.3', '0.00I', '2018-06-06 21:44:20']
['fdata', '-36.9986', '143.8953', '317.8', '13.8', '68', '44', '2018-06-06 21:44:20', '2018-06-06 21:44:20.230732']
['fdata', '-36.742', '144.8955', '312.1', '11.9', '52', '38', '2018-06-06 21:44:20', '2018-06-06 21:44:20.230994']
['fdata', '-36.9285', '143.9622', '316.4', '11.4', '66', '43', '2018-06-06 21:44:20', '2018-06-06 21:44:20.231242']
['fdata', '-36.6158', '145.4424', '374.9', '136.2', '100', '101', '2018-06-06 21:44:20', '2018-06-06 21:44:20.231452']
['fdata', '-36.9139', '143.9765', '315.4', '10.2', '54', '42', '2018-06-06 21:44:20', '2018-06-06 21:44:20.231925']
['cdata', '948720', '18', '54.6', '12', '25.1', '78.1*', '51.3', '0.12G', '2018-06-06 21:44:21']
['fdata', '-36.923', '143.8106', '318.9', '16.5', '70', '45', '2018-06-06 21:44:21', '2018-06-06 21:44:21.23431

-------------------------------------------
Time: 2018-06-06 21:45:00
-------------------------------------------
['cdata', '948720', '18', '55.6', '8.2', '12', '77.9*', '54.9', '0.00I', '2018-06-06 21:44:55']
['fdata', '-36.6605', '141.2603', '328.4', '28.8', '81', '55', '2018-06-06 21:44:55', '2018-06-06 21:44:55.450837']
['fdata', '-36.4115', '144.0755', '325.9', '21.8', '79', '52', '2018-06-06 21:44:55', '2018-06-06 21:44:55.451048']
['fdata', '-36.2375', '143.4093', '328.6', '25.2', '82', '55', '2018-06-06 21:44:55', '2018-06-06 21:44:55.451499']
['fdata', '-36.6833', '142.8296', '316.1', '11', '65', '42', '2018-06-06 21:44:55', '2018-06-06 21:44:55.451664']
['fdata', '-35.9326', '141.958', '339.9', '37.9', '78', '66', '2018-06-06 21:44:55', '2018-06-06 21:44:55.451982']
['cdata', '948720', '18', '53.8', '6.7', '14', '80.1*', '52.7*', '0.00I', '2018-06-06 21:44:56']
['fdata', '-36.4956', '143.7732', '329.6', '27.6', '82', '56', '2018-06-06 21:44:56', '2018-06-06 21:44:56.457657']
